In [1]:
# %%
# import utilitiy packages
import os, sys, gc, warnings, logging, shutil
import json, time, math

from glob import glob

# import the necessary MLTK APIs for model profiling
from mltk.core import profile_model

# determine GPU number
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # use "-1" for CPU
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # hide INFO and WARNING messages

# define paths to model files
MODELS_DIR = "models/"
MODEL_TF = MODELS_DIR + "model.pb"
MODEL_NO_QUANT_TFLITE = MODELS_DIR + "model_no_quant.tflite"
MODEL_TFLITE_MICRO = MODELS_DIR + "model.cc"
SEED = 7

os.makedirs(MODELS_DIR, exist_ok=True)

logging.disable(logging.WARNING)
logging.disable(logging.INFO)
warnings.filterwarnings("ignore")

# import basic libraries
import random
import dill

import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import keras

# Set a "seed" value, so we get the same random numbers each time we run this notebook for reproducible results.
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

from utils.data_loader import load_dataset
from utils.data_desc import AVAILABEL_DATASETS, CLS_DATASETS, REG_DATASETS
from utils import quantize_model, brief_profile_model, encode_image, time_token_log

from sklearn.metrics import accuracy_score
from sklearn.metrics import root_mean_squared_error

from openai import OpenAI
from configs import AVAILABLE_LLMs

from exp_prompts import (
    DATA_PROMPTS,
    complete_by_contexts,
    complete_by_values,
    complete_by_images,
    complete_mm_prompt,
)

2024-11-29 18:12:18.683199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 18:12:18.705958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 18:12:18.705997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 18:12:18.720688: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 18:12:19.896754: W tensorflow/com

In [5]:
# from agents import AgentManager

# query_type = ['text', 'time', 'image']
# data_name = 'FloodModeling'
# task = 'regression'
# llm = 'gpt-4o-mini'
# b=3
# c=3

# code_path =  f"Q{'_'.join(query_type)}_C{c}_B{b}"
# image_paths = [] if 'image' not in query_type else glob(f"ts_images/{data_name}/*.png")
# # Run MONAQ Agents
# monaq_agent = AgentManager(task=task, data_name=data_name, llm=llm, n_budgets=b, n_candidates=c, modality=query_type, image_paths=image_paths)
# if len(query_type) == 1:
#     prompt = {
#         "time": complete_by_values(data_name, task),
#         "text": complete_by_contexts(data_name),
#         "image": complete_by_images(data_name),
#     }[query_type[0]]
# else:
#     prompt = complete_mm_prompt(data_name, query_type, task)

# monaq_agent.initiate_chat(prompt)
    
# dirname = f"agent_objects/{llm}/{task}/{code_path}"
# os.makedirs(dirname, exist_ok=True)
# filename = f"{dirname}/{data_name}.pkl"
# with open(filename, 'wb') as f:
#     dill.dump(monaq_agent, f)

💬 You: I need a model to classify heartbeat signals, intended for deployment on an edge device with 1 MB of storage and 128 KB of RAM. Since this is a critical healthcare task, the model must be highly accurate while maintaining a very low inference latency of under 100 ms.

Additionally, please consider the following dataset features when building the model:
Classes:
- Normal
- Abnormal

Dimensions:
0. Heart sound recordings 


More contextual information is also provided below.
The task is to classify the nature of the heartbeat signal.
Heart sound recordings were sourced from several contributors around the world, collected at either a clinical or nonclinical environment, from both healthy subjects and pathological patients. Each series represent the amplitude of the signal over time. The heart sound recordings were collected from different locations on the body. The typical four locations are aortic area, pulmonic area, tricuspid area and mitral area, but could be one of nine diffe

🤴🏻 MONAQ Manager We have analyzed your requirements as follows!
{'task_description': 'The user requires a model to classify heartbeat signals into normal and abnormal categories. The model is intended for deployment on an edge device with limited resources, specifically 1 MB of storage and 128 KB of RAM. The task is critical for healthcare, necessitating high accuracy and low inference latency under 100 ms.', 'data_aspects': {'name': 'Heartbeat Signal Dataset', 'description': 'The dataset consists of heart sound recordings sourced globally from both clinical and nonclinical environments. It includes recordings from healthy subjects and patients with cardiac conditions.', 'features': 'The dataset contains time series data of heart sound amplitudes recorded at 2,000Hz. Each instance is truncated to a length of 18,530 timestamps. There are two classes: normal (110 instances) and abnormal (299 instances).', 'context': 'Recordings were collected from various body locations, typically the ao

In [11]:
import dill, glob
import pandas as pd

file_names = glob.glob("agent_objects/gpt-4o/regression/Qtime_text_image_C*_B3/*.pkl")

time_cost = {
    "INIT": [],
    "DESIGN_2": [],
    "BUILD_2": [],
    "EVAL_2": [],
    "VERIFY_2": [],
    "INST_2": [],
    "CODE_2": [],
}

money_cost = {
    "INIT": [],
    "DESIGN_2": [],
    "BUILD_2": [],
    "EVAL_2": [],
    "VERIFY_2": [],
    "INST_2": [],
    "CODE_2": [],
}


token_time = {
    "modality": [],
    "llm": [],
    "data_name": [],
    "task": [],
    "step": [],
    "time": [],
    "token": [],
    "cost": [],
}

for fname in file_names:
    _, llm, task, configs, data_name = fname.split("/")
    print(llm, task, configs, data_name.split(".")[0])
    with open(fname, "rb") as f:
        agent = dill.load(f)

    for k in list(time_cost.keys()):
        time_cost[k].append(agent.timer[k])
        if k.startswith('BUILD') or k.startswith('EVAL') or k.startswith('VERIFY'):
            money_cost[k].append(agent.money[ k + '_2']['prompt_tokens'] * in_cost + agent.money[ k + '_2']['completion_tokens'] * out_cost)
        else:
            money_cost[k].append(agent.money[k]['prompt_tokens'] * in_cost + agent.money[k]['completion_tokens'] * out_cost)

gpt-4o regression Qtime_text_image_C3_B3 AppliancesEnergy
gpt-4o regression Qtime_text_image_C3_B3 BIDMC32SpO2
gpt-4o regression Qtime_text_image_C3_B3 BenzeneConcentration


gpt-4o regression Qtime_text_image_C3_B3 FloodModeling
gpt-4o regression Qtime_text_image_C3_B3 LiveFuelMoistureContent


In [14]:
for key, value in money_cost.items():
    print(key, np.mean(value))

INIT 0.019610500000000003
DESIGN_2 0.012081000000000001
BUILD_2 0.013494
EVAL_2 0.0132835
VERIFY_2 0.0038710000000000003
INST_2 0.010948500000000002
CODE_2 0.017576


{'INIT': [0.034362500000000004,
  0.012955000000000001,
  0.019960000000000002,
  0.011745000000000002,
  0.019030000000000002],
 'DESIGN_2': [0.0120825,
  0.012382500000000001,
  0.012077500000000001,
  0.011962500000000001,
  0.0119],
 'BUILD_2': [0.013855,
  0.013427500000000002,
  0.014172500000000001,
  0.0127875,
  0.0132275],
 'EVAL_2': [0.013900000000000003,
  0.013392500000000002,
  0.013430000000000001,
  0.013070000000000002,
  0.012625],
 'VERIFY_2': [0.003985000000000001,
  0.0038075,
  0.0036175,
  0.0039025000000000006,
  0.0040425],
 'INST_2': [0.0101225,
  0.011335000000000001,
  0.014892500000000001,
  0.006302500000000001,
  0.012090000000000002],
 'CODE_2': [0.017535000000000002,
  0.01761,
  0.0181425,
  0.01716,
  0.017432500000000004]}